<a href="https://colab.research.google.com/github/yaman9675/IMDB-Dataset/blob/main/Yaman_Saini_M1W3D7_Guided_Project_EDA_on_IMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## You have been hired by a rookie movie producer to help him decide what type of movies to produce and which actors to cast. You have to back your recommendations based on thorough analysis of the data he shared with you which has the list of 3000 movies and the corresponding details.

## As a data scientist, you have to first explore the data and check its sanity.

## Further, you have to answer the following questions:
1. ### <b> Which movie made the highest profit? Who were its producer and director? Identify the actors in that film.</b>
2. ### <b>This data has information about movies made in different languages. Which language has the highest average ROI (return on investment)? </b>
3. ### <b> Find out the unique genres of movies in this dataset.</b>
4. ### <b> Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI? </b>
5. ### <b> Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor. </b>
6. ### <b>Top 3 directors prefer which actors the most? </b>



# Data Exploration

In [1]:
#Import package
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
path='/content/drive/MyDrive/Colab Notebooks/Module 1 : Python for Data Science/Yaman Dataset Module-1/Pandas & Data Wrangling/imdb_data.csv'
imdb_df = pd.read_csv(path)

In [4]:
imdb_df.head(2)

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435


In [ ]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

After reading all Qs. now subsetting out columns neceesary for answering the question in order to get proper insights

**keep all columns which are non null

In [ ]:
columns_to_keep= ['budget', 'genres','original_language', 'original_title','cast', 'crew', 'revenue']

In [ ]:
#find all the row indexes for which genres is not null
imdb_df.loc[~imdb_df['genres'].isna(),'genres']

In [ ]:
type(imdb_df.loc[0,'cast'])

str

Converting the string values to proper list and we are using it on columns --> cast crew genres

Converting only non null string

In [ ]:
def convert_to_list(str):
  return eval(str)

In [ ]:
#apply the above function only on non null values in genres column
imdb_df.loc[~imdb_df['genres'].isna(),'genres']= imdb_df.loc[~imdb_df['genres'].isna(),'genres'].apply(convert_to_list)

In [ ]:
#apply the above function only on non null values in cast column
imdb_df.loc[~imdb_df['cast'].isna(),'cast']= imdb_df.loc[~imdb_df['cast'].isna(),'cast'].apply(convert_to_list)

In [ ]:
#apply the above function only on non null values in crew column
imdb_df.loc[~imdb_df['crew'].isna(),'crew']= imdb_df.loc[~imdb_df['crew'].isna(),'crew'].apply(convert_to_list)

just keep in mind which column we have to use

In [ ]:
#creating a copy of orginal df
imdb_df_new = imdb_df.copy()

In [ ]:
imdb_df_new.head(2)

1) Which movie made the highest profit? Who were its producer and director? Identify the actors in that film.

In [ ]:
#checking for sanity in budget columns (outliers,vague values etc)
imdb_df_new.describe()
#budget of a movie in general cannot be 0 hence replacing those value with 0

In [ ]:
imdb_df_new[imdb_df_new['budget']==0].head(3)

In [ ]:
imdb_df_new['budget'].median()

8000000.0

In [ ]:
#Replace extremely low values of budget and revenue column with median values of budget, revenue
imdb_df_new.loc[imdb_df_new['budget']<1000,'budget']= imdb_df_new['budget'].median()

imdb_df_new.loc[imdb_df_new['revenue']<1000,'revenue']= imdb_df_new['revenue'].median()

In [ ]:
imdb_df_new.describe() #now fine

,id,budget,popularity,runtime,revenue
count,3000.000000,3.000000e+03,3000.000000,2998.000000,3.000000e+03
mean,1500.500000,2.474467e+07,8.463274,107.856571,6.704518e+07
std,866.169729,3.583254e+07,12.104000,22.086434,1.373964e+08
min,1.000000,2.500000e+03,0.000001,0.000000,1.404000e+03
25%,750.750000,8.000000e+06,4.018053,94.000000,2.947600e+06
50%,1500.500000,8.000000e+06,7.374861,104.000000,1.680873e+07
75%,2250.250000,2.900000e+07,10.890983,118.000000,6.891920e+07
max,3000.000000,3.800000e+08,294.337037,338.000000,1.519558e+09


In [ ]:
imdb_df_new['genres'].isnull().sum()

7

In [ ]:
#create profit and ROI column
imdb_df_new['profit'] = imdb_df_new['revenue'] - imdb_df_new['budget']
imdb_df_new['roi']= 100* (imdb_df_new['profit']/imdb_df_new['budget'])

In [ ]:
imdb_df_new.head(2)

In [ ]:
#maximum profit
imdb_df_new['profit'].max()

1316249360

In [ ]:
#find index or row which have the max profit using .idxmax()
#.idxmax()-->> returns the row number(index) for the max value of the column 
imdb_df_new['profit'].idxmax()

1761

The movie which made the highest profit is:

In [ ]:
imdb_df_new.loc[imdb_df_new['profit'].idxmax(),'original_title']

'Furious 7'

In [ ]:
max_profit_movie_df = imdb_df_new.iloc[imdb_df_new['profit'].idxmax()]

In [ ]:
max_profit_movie_df.head()

id                                                                    1762
belongs_to_collection    [{'id': 9485, 'name': 'The Fast and the Furiou...
budget                                                           190000000
genres                                      [{'id': 28, 'name': 'Action'}]
homepage                                          http://www.furious7.com/
Name: 1761, dtype: object

In [ ]:
max_profit_movie_df.loc['cast'][0]['name']

'Vin Diesel'

In [ ]:
crew_list= max_profit_movie_df.loc['crew']
crew_list[0:3]

Name of the director and producer of the movie which made highest profit are:

In [ ]:
producer_list=[]
director_list=[]
for elem in crew_list:
  if elem['job']=='Producer':
    producer_list.append(elem['name'])
  if elem['job']=='Director':
    director_list.append(elem['name'])

In [ ]:
print(f'PRODUCERS : {producer_list}')
print(f'DIRECTORS : {director_list}')

PRODUCERS : ['Vin Diesel', 'Neal H. Moritz', 'Michael Fottrell', 'Brandon Birtell']
DIRECTORS : ['James Wan']


In [ ]:
cast_list =max_profit_movie_df['cast']

In [ ]:
cast_list[0:3]

Actors in the Highest profit movie

In [ ]:
actor_list=[]
for elem in cast_list:
  actor_list.append(elem['name'])

In [ ]:
#actors
print(f'Actors of the movie are :')
actor_list

Actors of the movie are :


['Vin Diesel',
 'Paul Walker',
 'Dwayne Johnson',
 'Michelle Rodriguez',
 'Tyrese Gibson',
 'Ludacris',
 'Jordana Brewster',
 'Djimon Hounsou',
 'Tony Jaa',
 'Ronda Rousey',
 'Nathalie Emmanuel',
 'Kurt Russell',
 'Jason Statham',
 'Sung Kang',
 'Gal Gadot',
 'Lucas Black',
 'Elsa Pataky',
 'Noel Gugliemi',
 'John Brotherton',
 'Luke Evans',
 'Ali Fazal',
 'Miller Kimsey',
 'Charlie Kimsey',
 'Eden Estrella',
 'Gentry White',
 'Iggy Azalea',
 'Jon Lee Brody',
 'Levy Tran',
 'Anna Colwell',
 'Viktor Hernandez',
 'Steve Coulter',
 'Robert Pralgo',
 'Antwan Mills',
 'J.J. Phillips',
 'Jorge Ferragut',
 'Sara Sohn',
 'Benjamin Blankenship',
 'D.J. Hapa',
 'T-Pain',
 'Brian Mahoney',
 'Brittney Alger',
 'Romeo Santos',
 'Jocelin Donahue',
 'Stephanie Langston',
 'Jorge-Luis Pallo',
 'Tego Calder√≥n',
 'Nathalie Kelley',
 'Shad Moss',
 'Don Omar',
 'Klement Tinaj',
 'Caleb Walker',
 'Cody Walker']

2) This data has information about movies made in different languages. Which language has the highest average ROI (return on investment)?

In [ ]:
#Use groupby function on movie languages and ROI and finding mean
imdb_df_new.groupby('original_language')['roi'].mean().reset_index().sort_values(by='roi',ascending=False).head(3)

,original_language,roi
18,ko,11309.685605
6,el,5198.013245
28,sr,3261.413600


In [ ]:
print('Language with highest average roi is')
imdb_df_new.groupby('original_language')['roi'].mean().reset_index().sort_values(by='roi',ascending=False).iloc[0,0]

Language with highest average roi is


'ko'

3) Find out the unique genres of movies in this dataset.

In [ ]:
#considering only those rows in genres column which have no null values
no_na_genres = imdb_df_new[~imdb_df_new['genres'].isna()]

In [ ]:
len(no_na_genres)

2993

In [ ]:
no_na_genres.loc[0,'genres']

[{'id': 35, 'name': 'Comedy'}]

In [ ]:
no_na_genres.loc[0,'genres'][0]

{'id': 35, 'name': 'Comedy'}

In [ ]:
no_na_genres.loc[3,'genres']

[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]

In [ ]:
#create a list of genres and using .iterrow() method to iterate over genres column 
# .iterrow() --->> same as enumerate() its compulsory to use it in case of DataFrame
gen_list=[]
for index,row in no_na_genres.iterrows():
  genre = no_na_genres.loc[index,'genres']
  for k in genre:
    gen_list.append(k['name'])
  
#unique list of genres are:
pd.DataFrame(set(gen_list),columns=['Unique Genres'])

,Unique Genres
0,War
1,Foreign
2,Comedy
3,Romance
4,Mystery
5,Science Fiction
6,Thriller
7,Action
8,Drama
9,Music


4) Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI?

In [ ]:
#considering only those rows in crew column which have no null values
no_na_crew = imdb_df_new[~imdb_df_new['crew'].isna()]

In [ ]:
no_na_crew.shape

(2984, 25)

In [ ]:
#A simple function to extract list of all producers for a given movie_index
def create_producer_list(index):
  movie_index = no_na_crew.iloc[index]
  crew_list= movie_index.loc['crew']
  producer_list=[]
  for elem in crew_list:
     if elem['job']=='Producer':
        producer_list.append(elem['name'])
  return producer_list

In [ ]:
create_producer_list(61)

['James Hill']

In [ ]:
#A simple function to extract names of all directors for a given movie_index
#each movie has only one director
def create_director(index):
  movie_index = no_na_crew.iloc[index]
  crew_list= movie_index.loc['crew']

  for elem in crew_list:
     if elem['job']=='Director':
        return elem['name']

In [ ]:
create_director(61)

'Carol Reed'

In [ ]:
#create a empty DataFrame with required Column names in which we will append data later
Table = pd.DataFrame(columns=['Movie Title','Producers','Directors','ROI'])


,Movie Title,Producers,Directors,ROI


appending in Table Df and using Try Except block to bypass error because some values of the crew dictionaries contain float as value

In [ ]:
for index,row in no_na_crew.iterrows():
  
  try:
      Table = Table.append({'Movie Title':no_na_crew.loc[index,'original_title'],'Producers':create_producer_list(index),'Directors':create_director(index),'ROI':no_na_crew.loc[index,'roi']},ignore_index=True)
  except:
    continue

Table containing columns of Movie Title,its Producers

,Directors and ROI

In [ ]:
Table.head(10)

,Movie Title,Producers,Directors,ROI
0,Hot Tub Time Machine 2,[Andrew Panay],Steve Pink,-12.038207
1,The Princess Diaries 2: Royal Engagement,"[Whitney Houston, Mario Iscovich, Debra Martin...",Garry Marshall,137.873588
2,Whiplash,"[David Lancaster, Michel Litvak, Jason Blum, H...",Damien Chazelle,296.727273
3,Kahaani,[Sujoy Ghosh],Sujoy Ghosh,1233.333333
4,마린보이,[],Jong-seok Yoon,-50.950375
5,Pinocchio and the Emperor of the Night,[],Hal Sutherland,-59.229525
6,The Possession,[],Ole Bornedal,510.329107
7,Control Room,[],Jehane Noujaim,-67.668613
8,Muppet Treasure Island,"[Frank Oz, Brian Henson]",Brian Henson,329.092387
9,A Mighty Wind,[Karen Murphy],Christopher Guest,212.504100


5) Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor.

In [ ]:
#considering only those rows in cast column which have no null values
no_na_cast = imdb_df_new[~imdb_df_new['cast'].isna()]

In [ ]:
no_na_cast.loc[0,'cast'][0]['name']

'Rob Corddry'

In [ ]:
actor_list=[]
for index,row in no_na_cast.iterrows():
  for iter in no_na_cast.loc[index,'cast']:
    if type(iter)== dict:
      actor= iter['name']
      actor_list.append(actor)

In [ ]:
#create a  DataFrame with actor list 
Actor_Table = pd.DataFrame(actor_list,columns=['Name of Actor'])

In [ ]:
Actor_Table.shape

(61811, 1)

In [ ]:
Actor_Table.head()

,Name of Actor
0,Rob Corddry
1,Craig Robinson
2,Clark Duke
3,Adam Scott
4,Chevy Chase


In [ ]:
#sorting the actors using groupby function
Actor_Table.value_counts().reset_index().head()

,Name of Actor,0
0,Samuel L. Jackson,30
1,Robert De Niro,30
2,Morgan Freeman,27
3,Susan Sarandon,25
4,J.K. Simmons,25


Actors who have done maximum movies are:

In [ ]:
print('Samuel L. Jackson and Robert De Niro both have done 30 films')

Samuel L. Jackson and Robert De Niro both have done 30 films


In [ ]:

profit1=[]
profit2=[]
movie1=[]
movie2=[]
for index,row in no_na_cast.iterrows():
  for iter in no_na_cast.loc[index,'cast']:
    if type(iter)== dict:
      actor= iter['name']
      if 'Robert De Niro' in actor:
        profit1.append(no_na_cast.loc[index,'profit'])
        movie1.append(no_na_cast.loc[index,'original_title'])
           

      if 'Samuel L. Jackson' in actor:
        profit2.append(no_na_cast.loc[index,'profit'])
        movie2.append(no_na_cast.loc[index,'original_title'])

In [ ]:
#creating a loop to get the genres for Robert and Samuel
gener_r=[]
a=[]
for i in range(len(movie1)):
  for g in no_na_cast.loc[i,'genres']:
    a.append(g['name'])
    
  gener_r.append(a)
  a=[]

gener_s=[]
b=[]
for i in range(len(movie2)):
  for g in no_na_cast.loc[i,'genres']:
    b.append(g['name'])
    
  gener_s.append(b)
  b=[]

In [ ]:
genr = np.array(gener_r)
gens = np.array(gener_s)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
#creating sub dataframe for Robert
mov1= pd.DataFrame(movie1,columns=['Movie Name'])
prof1=pd.DataFrame(profit1,columns=['Movie Profit'])
gen1= pd.DataFrame(genr.flatten(),columns=['Genres'])

In [ ]:
Movies_by_Robert=pd.concat([mov1,gen1,prof1],axis=1)

Movies by Robert De Niro

In [ ]:
Movies_by_Robert.sort_values(by='Movie Profit',ascending=False).head()

,Movie Name,Genres,Movie Profit
8,Meet the Parents,"[Action, Comedy, Music, Family, Adventure]",275444045
16,American Hustle,"[Horror, Thriller]",211171807
10,Sleepers,[Drama],121615285
26,Analyze This,"[Crime, Drama, Mystery, Thriller]",96885658
29,Backdraft,"[Action, Adventure, Crime, Thriller]",77368585


In [ ]:
#creating sub dataframe for Samuel
mov2= pd.DataFrame(movie2,columns=['Movie Name'])
prof2=pd.DataFrame(profit2,columns=['Movie Profit'])
gen2= pd.DataFrame(gens.flatten(),columns=['Genres'])

In [ ]:
Movies_by_Samuel=pd.concat([mov1,gen1,prof1],axis=1)

Movies by Samuel L jackson

In [ ]:
Movies_by_Samuel.sort_values(by='Movie Profit',ascending=False).head()

,Movie Name,Genres,Movie Profit
8,Meet the Parents,"[Action, Comedy, Music, Family, Adventure]",275444045
16,American Hustle,"[Horror, Thriller]",211171807
10,Sleepers,[Drama],121615285
26,Analyze This,"[Crime, Drama, Mystery, Thriller]",96885658
29,Backdraft,"[Action, Adventure, Crime, Thriller]",77368585


6) Top 3 directors prefer which actors the most?

Top 3 Director who have directed the most movies

In [ ]:
directors = Table.loc[0:, 'Directors']

In [ ]:
directors.value_counts()

Clint Eastwood        11
Ron Howard            11
Steven Spielberg      10
Blake Edwards          9
Woody Allen            9
                      ..
Jonathan Barr√©        1
James Gartner          1
Jim Wynorski           1
Giuseppe Capotondi     1
David F. Sandberg      1
Name: Directors, Length: 1851, dtype: int64

In [ ]:
no_na_crew['directors'] = directors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dir_1 = no_na_crew[no_na_crew['directors'] == 'Ron Howard']
dir_2 = no_na_crew[no_na_crew['directors'] == 'Clint Eastwood']
dir_3 = no_na_crew[no_na_crew['directors'] == 'Steven Spielberg']

In [ ]:
def get_actors(dataframe):
 
  actor_list = []
  for i in range(len(dataframe)):
    try:
      for j in range((len(dataframe.loc[i, 'cast']))):
        try:
          actor_list.append(dataframe.loc[i, 'cast'][j]['name'])
        except:
          continue
    except:
          continue
  return actor_list

In [ ]:
dir_1.reset_index(inplace = True)

In [ ]:
dir_2.reset_index(inplace = True)

In [ ]:
dir_3.reset_index(inplace = True)

In [ ]:
get_actors(dir_1)

In [ ]:
get_actors(dir_2)

In [ ]:
get_actors(dir_3)